In [1]:
using FITSIO, StatsBase, Images, ImageView, JLD, Dates, FileIO, Formatting, Plots, FiniteDifferences

Gtk-Message: 20:40:47.008: Failed to load module "canberra-gtk-module"
Gtk-Message: 20:40:47.008: Failed to load module "canberra-gtk-module"


In [2]:
function remove_bad_pixels(data, threshold)
    d = zeros(Float64, size(data).+2)
    d[2:end-1, 2:end-1] = data
    out = (d[3:end, 2:end-1] + d[1:end-2, 2:end-1] + d[2:end-1, 3:end] + d[2:end-1, 1:end-2])/4
    out[out .< 0 ] .= 1
    t_out = copy(data)
    t_out[t_out .> threshold*out] = out[t_out .> threshold*out]
    return Float32.(t_out)
end

remove_bad_pixels (generic function with 1 method)

In [3]:
function crop_symmetric(data, n)
    return data[1+n:end-n, 1+n:end-n]
end

crop_symmetric (generic function with 1 method)

In [4]:
function normalize_vec(data, quant, value)
    p = quantile(vec(data), quant)
    return data.*(value/p)
end

function other_normalize_vec(data, quant, value)
    p = quantile(vec(data), quant)
    norm = mean(vec(data[data .<= p]))
    return data.*(value/norm)
end

other_normalize_vec (generic function with 1 method)

In [5]:
function import_fits(fname, crop::Int64=0)
    f = FITS(fname)
    data=read(f[2])
    data[data .< 0] .= 0
    temp = remove_bad_pixels(data, 25)
    temp = crop_symmetric(temp, crop)
    temp = normalize_vec(temp, 0.6, 10)
    return temp
end

function get_time(path)
    f_name = split(path, "/")[end]
    f_name = join(split(f_name, ".")[1:end-1])
    f_time = join(split(f_name, "_")[2:3])
    t = DateTime(f_time, "yyyymmddHHMM")
    return t
end

get_time (generic function with 1 method)

In [6]:
function convolve_circle(data, r_test)
    center = (size(data) .+ 1).*0.5
    r_test = 500
    tot = Float64(0)
    for i in eachindex(data)
        r = sqrt(sum((Tuple(i) .- center).^2))
        temp = 1 - abs(r - r_test)
        temp = (temp > 0.0) ? temp : 0.0
        tot += test_data[i]*temp
    end
    return tot
end

convolve_circle (generic function with 1 method)

In [18]:
function compute_circle_convolver(data, rs)
    data_size = size(data)
    center = (data_size .+ 1).*0.5
    convolver = Array{Float64, 3}(undef, data_size..., length(rs))
    for i in CartesianIndices(convolver)
        point = Tuple(i)
        r = sqrt(sum((point[1:2].-center).^2))
        temp = 1 - abs(rs[point[3]] - r)
        convolver[i] = (temp > 0.0) ? temp : 0.0
    end
    return convolver
end

function find_radius(data, min_r, max_r, r_step)
    rs = range(min_r, max_r, step=r_step)
    comp_circ = compute_circle_convolver(test_data, rs)
    convolves = sum(test_data .* comp_circ, dims=(1,2))
    convolves = convolves ./ sum(comp_circ, dims=(1,2))
    max, ind = findmax(convolves[:])
    return rs[ind]
end

find_radius (generic function with 2 methods)

In [22]:
test_file = "/data/R/solar_images/aia/synoptic/2014/05/18/AIA_20140518_1800_0335.fits"
test_data = import_fits(test_file)
find_radius(test_data, 300, 510, 0.5)

399.5

In [6]:
function import_data(filepath::String, wavelength::String, n::Int64=10000, crop::Int64=0)
    paths = []
    for (rootpath, dirs, files) in walkdir(filepath)
        for file in files
            if file[end-8:end-5]==wavelength
                append!(paths, [join([rootpath, file], "/")])
            end
        end
    end
    test_data = import_fits(paths[1])
    x,y = size(test_data)
    n = (length(paths)<n ? length(paths) : n)
    data = Array{Float32, 3}(undef, (x, y, n))
    times = Vector{DateTime}(undef, n)
    Threads.@threads for val in range(1, length=n)
        data[:,:,val] = import_fits(paths[val], crop)
        times[val] = get_time(paths[val])
    end
    return data, times
end

function import_data(filepath::String, n::Int64=10000, crop::Int64=0)
    paths = []
    for (rootpath, dirs, files) in walkdir(filepath)
        for file in files
            append!(paths, [join([rootpath, file], "/")])
        end
    end
    test_data = import_fits(paths[1])
    x,y = size(test_data)
    n = (length(paths)<n ? length(paths) : n)
    data = Array{Float32, 3}(undef, (x, y, n))
    times = Vector{DateTime}(undef, n)
    Threads.@threads for val in range(1, length=n)
        data[:,:,val] = import_fits(paths[val], crop)
        times[val] = get_time(paths[val])
    end
    return data, times
end

import_data (generic function with 6 methods)

In [7]:
function load_save_hmi_data(year)
    data, times = import_data(format("/data/R/solar_images/hmi/hmi_data/{:4d}", year))
    fname = format("./data/{:4d}_HMI.jld", year)
    save(File(format"JLD",fname),"data", data, "times", times)
end

function load_save_aia_data(year, wavelength)
    data, times = import_data(format("/data/R/solar_images/aia/synoptic/{:4d}", year), format("{:0>4d}", wavelength))
    fname = format("./data/{:4d}_AIA{:0>4d}.jld", year, wavelength)
    save(File(format"JLD",fname),"data", data, "times", times)
end

load_save_aia_data (generic function with 1 method)

In [ ]:
for year in range(2010, 2021, step=1)
    for wavelength in [94, 131, 171, 193, 211, 304, 335, 1600, 4500]
        load_save_aia_data(year, wavelength)
    end
    load_save_hmi_data(year)
end

In [68]:
temp_data = Array{Int64, 3}(undef, 2,3, 4)
for i in eachindex(temp_data)
    temp_data[i] = sum(Tuple(i))
end
print(temp_data)
temp_data_2 = [1 2 3; 1 2 3]

[1 3 5; 2 4 6]

[7 9 11; 8 10 12]

[13 15 17; 14 16 18]

[19 21 23; 20 22 24]

2×3 Matrix{Int64}:
 1  2  3
 1  2  3

In [69]:
temp_data_2.*temp_data

2×3×4 Array{Int64, 3}:
[:, :, 1] =
 1  6  15
 2  8  18

[:, :, 2] =
 7  18  33
 8  20  36

[:, :, 3] =
 13  30  51
 14  32  54

[:, :, 4] =
 19  42  69
 20  44  72

In [75]:
b = [1; 2; 3]

3-element Vector{Int64}:
 1
 2
 3

In [76]:
b[1:2]

2-element Vector{Int64}:
 1
 2

In [91]:
for i in CartesianIndices(temp_data)
    print(i)
end

CartesianIndex(1, 1, 1)CartesianIndex(2, 1, 1)CartesianIndex(1, 2, 1)CartesianIndex(2, 2, 1)CartesianIndex(1, 3, 1)CartesianIndex(2, 3, 1)CartesianIndex(1, 1, 2)CartesianIndex(2, 1, 2)CartesianIndex(1, 2, 2)CartesianIndex(2, 2, 2)CartesianIndex(1, 3, 2)CartesianIndex(2, 3, 2)CartesianIndex(1, 1, 3)CartesianIndex(2, 1, 3)CartesianIndex(1, 2, 3)CartesianIndex(2, 2, 3)CartesianIndex(1, 3, 3)CartesianIndex(2, 3, 3)CartesianIndex(1, 1, 4)CartesianIndex(2, 1, 4)CartesianIndex(1, 2, 4)CartesianIndex(2, 2, 4)CartesianIndex(1, 3, 4)CartesianIndex(2, 3, 4)